# Import the nessasary librarys

In [1]:
import numpy as np
import os
import time
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
import scipy.sparse as sp
from sklearn.model_selection import train_test_split

# The function for the evaluate.py


In [2]:
def hit(gt_item, pred_items):
	if gt_item in pred_items:
		return 1
	return 0


def ndcg(gt_item, pred_items):
	if gt_item in pred_items:
		index = pred_items.index(gt_item)
		return np.reciprocal(np.log2(index+2))
	return 0


def metrics(model, test_loader, top_k):
	HR, NDCG = [], []

	for user, item, label in test_loader:
		user = user.cuda()
		item = item.cuda()

		predictions = model(user, item)
		_, indices = torch.topk(predictions, top_k)
		recommends = torch.take(
				item, indices).cpu().numpy().tolist()

		gt_item = item[0].item()
		HR.append(hit(gt_item, recommends))
		NDCG.append(ndcg(gt_item, recommends))

	return np.mean(HR), np.mean(NDCG)

# The code for the model.py

In [3]:
class NCF(nn.Module):
	def __init__(self, user_num, item_num, factor_num, num_layers,
					dropout, model, GMF_model=None, MLP_model=None):
		super(NCF, self).__init__()
		"""
		user_num: number of users;
		item_num: number of items;
		factor_num: number of predictive factors;
		num_layers: the number of layers in MLP model;
		dropout: dropout rate between fully connected layers;
		model: 'MLP', 'GMF', 'NeuMF-end', and 'NeuMF-pre';
		GMF_model: pre-trained GMF weights;
		MLP_model: pre-trained MLP weights.
		"""		
		self.dropout = dropout
		self.model = model
		self.GMF_model = GMF_model
		self.MLP_model = MLP_model

		self.embed_user_GMF = nn.Embedding(user_num, factor_num)
		self.embed_item_GMF = nn.Embedding(item_num, factor_num)
		self.embed_user_MLP = nn.Embedding(
				user_num, factor_num * (2 ** (num_layers - 1)))
		self.embed_item_MLP = nn.Embedding(
				item_num, factor_num * (2 ** (num_layers - 1)))

		MLP_modules = []
		for i in range(num_layers):
			input_size = factor_num * (2 ** (num_layers - i))
			MLP_modules.append(nn.Dropout(p=self.dropout))
			MLP_modules.append(nn.Linear(input_size, input_size//2))
			MLP_modules.append(nn.ReLU())
		self.MLP_layers = nn.Sequential(*MLP_modules)

		if self.model in ['MLP', 'GMF']:
			predict_size = factor_num 
		else:
			predict_size = factor_num * 2
		self.predict_layer = nn.Linear(predict_size, 1)

		self._init_weight_()

	def _init_weight_(self):
		""" We leave the weights initialization here. """
		if not self.model == 'NeuMF-pre':
			nn.init.normal_(self.embed_user_GMF.weight, std=0.01)
			nn.init.normal_(self.embed_user_MLP.weight, std=0.01)
			nn.init.normal_(self.embed_item_GMF.weight, std=0.01)
			nn.init.normal_(self.embed_item_MLP.weight, std=0.01)

			for m in self.MLP_layers:
				if isinstance(m, nn.Linear):
					nn.init.xavier_uniform_(m.weight)
			nn.init.kaiming_uniform_(self.predict_layer.weight, 
									a=1, nonlinearity='sigmoid')

			for m in self.modules():
				if isinstance(m, nn.Linear) and m.bias is not None:
					m.bias.data.zero_()
		else:
			# embedding layers
			self.embed_user_GMF.weight.data.copy_(
							self.GMF_model.embed_user_GMF.weight)
			self.embed_item_GMF.weight.data.copy_(
							self.GMF_model.embed_item_GMF.weight)
			self.embed_user_MLP.weight.data.copy_(
							self.MLP_model.embed_user_MLP.weight)
			self.embed_item_MLP.weight.data.copy_(
							self.MLP_model.embed_item_MLP.weight)

			# mlp layers
			for (m1, m2) in zip(
				self.MLP_layers, self.MLP_model.MLP_layers):
				if isinstance(m1, nn.Linear) and isinstance(m2, nn.Linear):
					m1.weight.data.copy_(m2.weight)
					m1.bias.data.copy_(m2.bias)

			# predict layers
			predict_weight = torch.cat([
				self.GMF_model.predict_layer.weight, 
				self.MLP_model.predict_layer.weight], dim=1)
			precit_bias = self.GMF_model.predict_layer.bias + \
						self.MLP_model.predict_layer.bias

			self.predict_layer.weight.data.copy_(0.5 * predict_weight)
			self.predict_layer.bias.data.copy_(0.5 * precit_bias)

	def forward(self, user, item):
		if not self.model == 'MLP':
			embed_user_GMF = self.embed_user_GMF(user)
			embed_item_GMF = self.embed_item_GMF(item)
			output_GMF = embed_user_GMF * embed_item_GMF
		if not self.model == 'GMF':
			embed_user_MLP = self.embed_user_MLP(user)
			embed_item_MLP = self.embed_item_MLP(item)
			interaction = torch.cat((embed_user_MLP, embed_item_MLP), -1)
			output_MLP = self.MLP_layers(interaction)

		if self.model == 'GMF':
			concat = output_GMF
		elif self.model == 'MLP':
			concat = output_MLP
		else:
			concat = torch.cat((output_GMF, output_MLP), -1)

		prediction = self.predict_layer(concat)
		return prediction.view(-1)

# ***Must to be chance***

# The code for the config.py

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# dataset name 
dataset = 'u.data'

# model name 
model = 'NeuMF-end'

%cd 'drive/MyDrive/Colab Notebooks/deep'
model_path = 'model/'

train_rating = '{}'.format(dataset)
test_negative = '{}.negative'.format(dataset)

/content/drive/MyDrive/Colab Notebooks/deep


# The code for the data_utils.py

In [6]:
def load_all(test_num=100):
    """ Load all three files to save time in each epoch. """
    train_data = pd.read_csv(
        train_rating,
        sep='\t', header=None, names=['user', 'item'],
        usecols=[0, 1], dtype={0: np.int32, 1: np.int32})

    user_num = train_data['user'].max() + 1
    item_num = train_data['item'].max() + 1

    train_x, test_x, train_y, test_y = train_test_split(train_data['user'], train_data['item'], test_size=0.2,
                                                        random_state=42)
    train_data = train_data.loc[train_data['user'].isin(train_x) & train_data['item'].isin(train_y)]
    test_data = pd.DataFrame({'user': test_x, 'item': test_y})

    train_data = train_data.values.tolist()
    test_data = test_data.values.tolist()

    # Load ratings as a dok matrix
    train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    for x in train_data:
        train_mat[x[0], x[1]] = 1.0

    return train_data, test_data, user_num, item_num, train_mat


class NCFData(data.Dataset):
	def __init__(self, features, 
				num_item, train_mat=None, num_ng=0, is_training=None):
		super(NCFData, self).__init__()
		""" Note that the labels are only useful when training, we thus 
			add them in the ng_sample() function.
		"""
		self.features_ps = features
		self.num_item = num_item
		self.train_mat = train_mat
		self.num_ng = num_ng
		self.is_training = is_training
		self.labels = [0 for _ in range(len(features))]

	def ng_sample(self):
		assert self.is_training, 'no need to sampling when testing'

		self.features_ng = []
		for x in self.features_ps:
			u = x[0]
			for t in range(self.num_ng):
				j = np.random.randint(self.num_item)
				while (u, j) in self.train_mat:
					j = np.random.randint(self.num_item)
				self.features_ng.append([u, j])

		labels_ps = [1 for _ in range(len(self.features_ps))]
		labels_ng = [0 for _ in range(len(self.features_ng))]

		self.features_fill = self.features_ps + self.features_ng
		self.labels_fill = labels_ps + labels_ng

	def __len__(self):
		return (self.num_ng + 1) * len(self.labels)

	def __getitem__(self, idx):
		features = self.features_fill if self.is_training \
					else self.features_ps
		labels = self.labels_fill if self.is_training \
					else self.labels

		user = features[idx][0]
		item = features[idx][1]
		label = labels[idx]
		return user, item ,label

# The code for the main.py

## The default argument


In [7]:
lr =0.001
dropout=0.0
batch_size=256
epochs=20
# top_k=10 
factor_num=32
num_layers=3 
num_ng=4
test_num_ng=99
out=True
gpu='0'

os.environ["CUDA_VISIBLE_DEVICES"] = gpu
cudnn.benchmark = True

## The code

In [8]:
for top_k in range(1,11):
	print("Top_K",top_k)
	############################## PREPARE DATASET ##########################
	train_data, test_data, user_num ,item_num, train_mat = load_all()

	# construct the train and test datasets
	train_dataset = NCFData(
			train_data, item_num, train_mat, num_ng, True)
	test_dataset = NCFData(
			test_data, item_num, train_mat, 0, False)
	train_loader = data.DataLoader(train_dataset,
			batch_size=batch_size, shuffle=True, num_workers=4)
	test_loader = data.DataLoader(test_dataset,
			batch_size=test_num_ng+1, shuffle=False, num_workers=0)

	########################### CREATE MODEL #################################
	GMF_model = None
	MLP_model = None

	model = NCF(user_num, item_num, factor_num, num_layers, dropout, model, GMF_model, MLP_model)
	model.cuda()
	loss_function = nn.BCEWithLogitsLoss()


	optimizer = optim.Adam(model.parameters(), lr=lr)

	# writer = SummaryWriter() # for visualization

	########################### TRAINING #####################################
	count, best_hr = 0, 0
	for epoch in range(epochs):
		model.train() # Enable dropout (if have).
		start_time = time.time()
		train_loader.dataset.ng_sample()

		for user, item, label in train_loader:
			user = user.cuda()
			item = item.cuda()
			label = label.float().cuda()

			model.zero_grad()
			prediction = model(user, item)
			loss = loss_function(prediction, label)
			loss.backward()
			optimizer.step()
			# writer.add_scalar('data/loss', loss.item(), count)
			count += 1

		model.eval()
		HR, NDCG = metrics(model, test_loader, top_k)

		elapsed_time = time.time() - start_time
		print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
				time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
		print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

		if HR > best_hr:
			best_hr, best_ndcg, best_epoch, best_model = HR, NDCG, epoch, model
			if out:
				if not os.path.exists(model_path):
					os.mkdir(model_path)
				# torch.save(model, '{}{}.pth'.format(model_path, model))

	print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))

Top_K 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 14
HR: 0.010	NDCG: 0.010
The time elapse of epoch 001 is: 00: 00: 13
HR: 0.010	NDCG: 0.010
The time elapse of epoch 002 is: 00: 00: 12
HR: 0.015	NDCG: 0.015
The time elapse of epoch 003 is: 00: 00: 12
HR: 0.005	NDCG: 0.005
The time elapse of epoch 004 is: 00: 00: 12
HR: 0.010	NDCG: 0.010
The time elapse of epoch 005 is: 00: 00: 12
HR: 0.010	NDCG: 0.010
The time elapse of epoch 006 is: 00: 00: 11
HR: 0.005	NDCG: 0.005
The time elapse of epoch 007 is: 00: 00: 11
HR: 0.005	NDCG: 0.005
The time elapse of epoch 008 is: 00: 00: 11
HR: 0.010	NDCG: 0.010
The time elapse of epoch 009 is: 00: 00: 12
HR: 0.010	NDCG: 0.010
The time elapse of epoch 010 is: 00: 00: 12
HR: 0.005	NDCG: 0.005
The time elapse of epoch 011 is: 00: 00: 12
HR: 0.000	NDCG: 0.000
The time elapse of epoch 012 is: 00: 00: 12
HR: 0.000	NDCG: 0.000
The time elapse of epoch 013 is: 00: 00: 12
HR: 0.000	NDCG: 0.000
The time elapse of epoch 014 is: 00: 00: 12
HR: 0.005	NDCG: 0.005
The time e

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 12
HR: 0.030	NDCG: 0.026
The time elapse of epoch 001 is: 00: 00: 12
HR: 0.025	NDCG: 0.021
The time elapse of epoch 002 is: 00: 00: 12
HR: 0.020	NDCG: 0.018
The time elapse of epoch 003 is: 00: 00: 12
HR: 0.020	NDCG: 0.014
The time elapse of epoch 004 is: 00: 00: 12
HR: 0.015	NDCG: 0.013
The time elapse of epoch 005 is: 00: 00: 12
HR: 0.010	NDCG: 0.008
The time elapse of epoch 006 is: 00: 00: 12
HR: 0.015	NDCG: 0.013
The time elapse of epoch 007 is: 00: 00: 12
HR: 0.010	NDCG: 0.008
The time elapse of epoch 008 is: 00: 00: 11
HR: 0.005	NDCG: 0.003
The time elapse of epoch 009 is: 00: 00: 12
HR: 0.010	NDCG: 0.006
The time elapse of epoch 010 is: 00: 00: 12
HR: 0.015	NDCG: 0.009
The time elapse of epoch 011 is: 00: 00: 12
HR: 0.005	NDCG: 0.003
The time elapse of epoch 012 is: 00: 00: 12
HR: 0.015	NDCG: 0.009
The time elapse of epoch 013 is: 00: 00: 12
HR: 0.015	NDCG: 0.015
The time elapse of epoch 014 is: 00: 00: 12
HR: 0.015	NDCG: 0.011
The time e

...

The time elapse of epoch 000 is: 00: 00: 13
HR: 0.095	NDCG: 0.049
The time elapse of epoch 001 is: 00: 00: 13
HR: 0.125	NDCG: 0.063
The time elapse of epoch 002 is: 00: 00: 13
HR: 0.100	NDCG: 0.048
The time elapse of epoch 003 is: 00: 00: 13
HR: 0.105	NDCG: 0.052
The time elapse of epoch 004 is: 00: 00: 14
HR: 0.100	NDCG: 0.048
The time elapse of epoch 005 is: 00: 00: 13
HR: 0.100	NDCG: 0.054
The time elapse of epoch 006 is: 00: 00: 13
HR: 0.105	NDCG: 0.049
The time elapse of epoch 007 is: 00: 00: 13
HR: 0.090	NDCG: 0.043
The time elapse of epoch 008 is: 00: 00: 13
HR: 0.095	NDCG: 0.044
The time elapse of epoch 009 is: 00: 00: 13
HR: 0.075	NDCG: 0.034
The time elapse of epoch 010 is: 00: 00: 13
HR: 0.090	NDCG: 0.039
The time elapse of epoch 011 is: 00: 00: 13
HR: 0.105	NDCG: 0.041
The time elapse of epoch 012 is: 00: 00: 13
HR: 0.075	NDCG: 0.030
The time elapse of epoch 013 is: 00: 00: 13
HR: 0.080	NDCG: 0.034
The time elapse of epoch 014 is: 00: 00: 13
HR: 0.070	NDCG: 0.029
The time e

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 13
HR: 0.095	NDCG: 0.048
The time elapse of epoch 001 is: 00: 00: 13
HR: 0.105	NDCG: 0.046
The time elapse of epoch 002 is: 00: 00: 13
HR: 0.105	NDCG: 0.047
The time elapse of epoch 003 is: 00: 00: 13
HR: 0.110	NDCG: 0.049
The time elapse of epoch 004 is: 00: 00: 13
HR: 0.105	NDCG: 0.049
The time elapse of epoch 005 is: 00: 00: 13
HR: 0.100	NDCG: 0.046
The time elapse of epoch 006 is: 00: 00: 13
HR: 0.095	NDCG: 0.045
The time elapse of epoch 007 is: 00: 00: 13
HR: 0.095	NDCG: 0.045
The time elapse of epoch 008 is: 00: 00: 14
HR: 0.110	NDCG: 0.049
The time elapse of epoch 009 is: 00: 00: 13
HR: 0.125	NDCG: 0.050
The time elapse of epoch 010 is: 00: 00: 13
HR: 0.115	NDCG: 0.047
The time elapse of epoch 011 is: 00: 00: 13
HR: 0.095	NDCG: 0.042
The time elapse of epoch 012 is: 00: 00: 13
HR: 0.130	NDCG: 0.053
The time elapse of epoch 013 is: 00: 00: 13
HR: 0.110	NDCG: 0.047
The time elapse of epoch 014 is: 00: 00: 13
HR: 0.120	NDCG: 0.050
The time e

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 13
HR: 0.085	NDCG: 0.042
The time elapse of epoch 001 is: 00: 00: 13
HR: 0.100	NDCG: 0.045
The time elapse of epoch 002 is: 00: 00: 13
HR: 0.115	NDCG: 0.048
The time elapse of epoch 003 is: 00: 00: 13
HR: 0.115	NDCG: 0.052
The time elapse of epoch 004 is: 00: 00: 13
HR: 0.120	NDCG: 0.052
The time elapse of epoch 005 is: 00: 00: 13
HR: 0.120	NDCG: 0.054
The time elapse of epoch 006 is: 00: 00: 13
HR: 0.115	NDCG: 0.049
The time elapse of epoch 007 is: 00: 00: 13
HR: 0.090	NDCG: 0.042
The time elapse of epoch 008 is: 00: 00: 13
HR: 0.080	NDCG: 0.038
The time elapse of epoch 009 is: 00: 00: 13
HR: 0.095	NDCG: 0.041
The time elapse of epoch 010 is: 00: 00: 13
HR: 0.105	NDCG: 0.045
The time elapse of epoch 011 is: 00: 00: 13
HR: 0.090	NDCG: 0.038
The time elapse of epoch 012 is: 00: 00: 13
HR: 0.075	NDCG: 0.031
The time elapse of epoch 013 is: 00: 00: 13
HR: 0.085	NDCG: 0.034
The time elapse of epoch 014 is: 00: 00: 13
HR: 0.085	NDCG: 0.035
The time e

In [9]:
top_k=10 
for num_ng in range(1,11):
	print("Num_ng",num_ng)
	############################## PREPARE DATASET ##########################
	train_data, test_data, user_num ,item_num, train_mat = load_all()

	# construct the train and test datasets
	train_dataset = NCFData(
			train_data, item_num, train_mat, num_ng, True)
	test_dataset = NCFData(
			test_data, item_num, train_mat, 0, False)
	train_loader = data.DataLoader(train_dataset,
			batch_size=batch_size, shuffle=True, num_workers=4)
	test_loader = data.DataLoader(test_dataset,
			batch_size=test_num_ng+1, shuffle=False, num_workers=0)

	########################### CREATE MODEL #################################
	GMF_model = None
	MLP_model = None

	model = NCF(user_num, item_num, factor_num, num_layers, dropout, model, GMF_model, MLP_model)
	model.cuda()
	loss_function = nn.BCEWithLogitsLoss()


	optimizer = optim.Adam(model.parameters(), lr=lr)

	# writer = SummaryWriter() # for visualization

	########################### TRAINING #####################################
	count, best_hr = 0, 0
	for epoch in range(epochs):
		model.train() # Enable dropout (if have).
		start_time = time.time()
		train_loader.dataset.ng_sample()

		for user, item, label in train_loader:
			user = user.cuda()
			item = item.cuda()
			label = label.float().cuda()

			model.zero_grad()
			prediction = model(user, item)
			loss = loss_function(prediction, label)
			loss.backward()
			optimizer.step()
			# writer.add_scalar('data/loss', loss.item(), count)
			count += 1

		model.eval()
		HR, NDCG = metrics(model, test_loader, top_k)

		elapsed_time = time.time() - start_time
		print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
				time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
		print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

		if HR > best_hr:
			best_hr, best_ndcg, best_epoch, best_model = HR, NDCG, epoch, model
			if out:
				if not os.path.exists(model_path):
					os.mkdir(model_path)
				# torch.save(model, '{}{}.pth'.format(model_path, model))

	print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))

Num_ng 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 05
HR: 0.085	NDCG: 0.038
The time elapse of epoch 001 is: 00: 00: 04
HR: 0.095	NDCG: 0.045
The time elapse of epoch 002 is: 00: 00: 06
HR: 0.090	NDCG: 0.050
The time elapse of epoch 003 is: 00: 00: 04
HR: 0.105	NDCG: 0.045
The time elapse of epoch 004 is: 00: 00: 05
HR: 0.105	NDCG: 0.049
The time elapse of epoch 005 is: 00: 00: 05
HR: 0.115	NDCG: 0.051
The time elapse of epoch 006 is: 00: 00: 04
HR: 0.105	NDCG: 0.049
The time elapse of epoch 007 is: 00: 00: 05
HR: 0.090	NDCG: 0.050
The time elapse of epoch 008 is: 00: 00: 04
HR: 0.120	NDCG: 0.058
The time elapse of epoch 009 is: 00: 00: 05
HR: 0.115	NDCG: 0.054
The time elapse of epoch 010 is: 00: 00: 05
HR: 0.100	NDCG: 0.051
The time elapse of epoch 011 is: 00: 00: 05
HR: 0.080	NDCG: 0.042
The time elapse of epoch 012 is: 00: 00: 05
HR: 0.085	NDCG: 0.039
The time elapse of epoch 013 is: 00: 00: 05
HR: 0.080	NDCG: 0.041
The time elapse of epoch 014 is: 00: 00: 05
HR: 0.070	NDCG: 0.035
The time e

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 07
HR: 0.105	NDCG: 0.049
The time elapse of epoch 001 is: 00: 00: 08
HR: 0.095	NDCG: 0.045
The time elapse of epoch 002 is: 00: 00: 07
HR: 0.100	NDCG: 0.049
The time elapse of epoch 003 is: 00: 00: 08
HR: 0.105	NDCG: 0.050
The time elapse of epoch 004 is: 00: 00: 07
HR: 0.120	NDCG: 0.055
The time elapse of epoch 005 is: 00: 00: 08
HR: 0.105	NDCG: 0.049
The time elapse of epoch 006 is: 00: 00: 08
HR: 0.105	NDCG: 0.049
The time elapse of epoch 007 is: 00: 00: 07
HR: 0.100	NDCG: 0.046
The time elapse of epoch 008 is: 00: 00: 08
HR: 0.100	NDCG: 0.046
The time elapse of epoch 009 is: 00: 00: 07
HR: 0.125	NDCG: 0.054
The time elapse of epoch 010 is: 00: 00: 08
HR: 0.110	NDCG: 0.051
The time elapse of epoch 011 is: 00: 00: 08
HR: 0.100	NDCG: 0.044
The time elapse of epoch 012 is: 00: 00: 07
HR: 0.095	NDCG: 0.042
The time elapse of epoch 013 is: 00: 00: 08
HR: 0.090	NDCG: 0.038
The time elapse of epoch 014 is: 00: 00: 07
HR: 0.115	NDCG: 0.045
The time e

...

The time elapse of epoch 000 is: 00: 00: 25
HR: 0.105	NDCG: 0.056
The time elapse of epoch 001 is: 00: 00: 25
HR: 0.130	NDCG: 0.069
The time elapse of epoch 002 is: 00: 00: 25
HR: 0.120	NDCG: 0.053
The time elapse of epoch 003 is: 00: 00: 25
HR: 0.105	NDCG: 0.044
The time elapse of epoch 004 is: 00: 00: 25
HR: 0.110	NDCG: 0.044
The time elapse of epoch 005 is: 00: 00: 24
HR: 0.100	NDCG: 0.039
The time elapse of epoch 006 is: 00: 00: 24
HR: 0.125	NDCG: 0.048
The time elapse of epoch 007 is: 00: 00: 26
HR: 0.115	NDCG: 0.046
The time elapse of epoch 008 is: 00: 00: 25
HR: 0.125	NDCG: 0.048
The time elapse of epoch 009 is: 00: 00: 25
HR: 0.130	NDCG: 0.054
The time elapse of epoch 010 is: 00: 00: 25
HR: 0.130	NDCG: 0.054
The time elapse of epoch 011 is: 00: 00: 26
HR: 0.125	NDCG: 0.051
The time elapse of epoch 012 is: 00: 00: 24
HR: 0.115	NDCG: 0.049
The time elapse of epoch 013 is: 00: 00: 24
HR: 0.100	NDCG: 0.042
The time elapse of epoch 014 is: 00: 00: 25
HR: 0.100	NDCG: 0.044
The time e

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 28
HR: 0.110	NDCG: 0.049
The time elapse of epoch 001 is: 00: 00: 28
HR: 0.105	NDCG: 0.047
The time elapse of epoch 002 is: 00: 00: 28
HR: 0.105	NDCG: 0.051
The time elapse of epoch 003 is: 00: 00: 28
HR: 0.120	NDCG: 0.051
The time elapse of epoch 004 is: 00: 00: 28
HR: 0.110	NDCG: 0.045
The time elapse of epoch 005 is: 00: 00: 28
HR: 0.125	NDCG: 0.051
The time elapse of epoch 006 is: 00: 00: 28
HR: 0.100	NDCG: 0.041
The time elapse of epoch 007 is: 00: 00: 28
HR: 0.095	NDCG: 0.037
The time elapse of epoch 008 is: 00: 00: 27
HR: 0.110	NDCG: 0.046
The time elapse of epoch 009 is: 00: 00: 28
HR: 0.105	NDCG: 0.041
The time elapse of epoch 010 is: 00: 00: 28
HR: 0.120	NDCG: 0.051
The time elapse of epoch 011 is: 00: 00: 28
HR: 0.095	NDCG: 0.040
The time elapse of epoch 012 is: 00: 00: 28
HR: 0.110	NDCG: 0.045
The time elapse of epoch 013 is: 00: 00: 28
HR: 0.100	NDCG: 0.043
The time elapse of epoch 014 is: 00: 00: 29
HR: 0.090	NDCG: 0.040
The time e

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 31
HR: 0.110	NDCG: 0.052
The time elapse of epoch 001 is: 00: 00: 31
HR: 0.125	NDCG: 0.055
The time elapse of epoch 002 is: 00: 00: 32
HR: 0.135	NDCG: 0.062
The time elapse of epoch 003 is: 00: 00: 32
HR: 0.140	NDCG: 0.058
The time elapse of epoch 004 is: 00: 00: 31
HR: 0.115	NDCG: 0.051
The time elapse of epoch 005 is: 00: 00: 32
HR: 0.115	NDCG: 0.051
The time elapse of epoch 006 is: 00: 00: 31
HR: 0.110	NDCG: 0.046
The time elapse of epoch 007 is: 00: 00: 31
HR: 0.100	NDCG: 0.042
The time elapse of epoch 008 is: 00: 00: 32
HR: 0.130	NDCG: 0.052
The time elapse of epoch 009 is: 00: 00: 31
HR: 0.115	NDCG: 0.046
The time elapse of epoch 010 is: 00: 00: 31
HR: 0.115	NDCG: 0.046
The time elapse of epoch 011 is: 00: 00: 31
HR: 0.115	NDCG: 0.048
The time elapse of epoch 012 is: 00: 00: 32
HR: 0.115	NDCG: 0.047
The time elapse of epoch 013 is: 00: 00: 31
HR: 0.125	NDCG: 0.049
The time elapse of epoch 014 is: 00: 00: 31
HR: 0.130	NDCG: 0.051
The time e